In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D

import numpy as np
import pandas as pd

# set parameters:
max_features = 5000
batch_size = 32
embedding_dims = 20
filters = 250
kernel_size = 3
hidden_dims = 250

train = pd.read_csv('train.csv')
print(train.head())
print("training data shape:", train.shape) # three columns are id, text, author

from keras.utils import to_categorical
a2c = {'EAP': 0, 'HPL' : 1, 'MWS' : 2}
labels = np.array([a2c[a] for a in train.author])
labels = to_categorical(labels)
print("train labels: ", labels)

text = train["text"]
print("train text shape: ", text.shape)

from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(filters="", lower=True, split=" ", char_level=False)
tokenizer.fit_on_texts(text);
docs = tokenizer.texts_to_sequences(text);
print("text: ", text[0])
print("tokenized: ", docs[0])


maxlen = np.amax([len(x) for x in docs], axis=0)
print("max doc length: ", maxlen)

print('Pad sequences (samples x time)')
docs = sequence.pad_sequences(docs, maxlen=maxlen)
print('padded docs shape:', docs.shape)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(docs, labels, test_size=0.2)

Using TensorFlow backend.


        id                                               text author
0  id26305  This process, however, afforded me no means of...    EAP
1  id17569  It never once occurred to me that the fumbling...    HPL
2  id11008  In his left hand was a gold snuff box, from wh...    EAP
3  id27763  How lovely is spring As we looked from Windsor...    MWS
4  id12958  Finding nothing else, not even gold, the Super...    HPL
training data shape: (19579, 3)
train labels:  [[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 1.  0.  0.]
 ..., 
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]]
train text shape:  (19579,)
text:  This process, however, afforded me no means of ascertaining the dimensions of my dungeon; as I might make its circuit, and return to the point whence I set out, without being aware of the fact; so perfectly uniform seemed the wall.
tokenized:  [26, 8207, 142, 1331, 31, 37, 272, 2, 12060, 1, 5088, 2, 10, 22483, 16, 6, 75, 160, 45, 22484, 3, 341, 4, 1, 274, 1996, 6, 302, 1062, 121, 123, 886, 2, 1, 22485, 

In [2]:
model = Sequential()
input_dim = np.max(docs) + 1
# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(input_dim,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))



model.add(Dense(3, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


from keras.callbacks import EarlyStopping
model.fit(x_train, y_train,
                 batch_size=16,
                 validation_data=(x_test, y_test),
                 epochs=10,
                 callbacks=[EarlyStopping(patience=2, monitor='val_loss')])

Train on 15663 samples, validate on 3916 samples
Epoch 1/10
15663/15663 [==============================] - 14s 891us/step - loss: 0.4917 - acc: 0.7605 - val_loss: 0.3424 - val_acc: 0.8532
Epoch 2/10
15663/15663 [==============================] - 12s 756us/step - loss: 0.2334 - acc: 0.9068 - val_loss: 0.3191 - val_acc: 0.8606
Epoch 3/10
15663/15663 [==============================] - 11s 731us/step - loss: 0.0978 - acc: 0.9654 - val_loss: 0.3462 - val_acc: 0.8704
Epoch 4/10
15663/15663 [==============================] - 11s 724us/step - loss: 0.0428 - acc: 0.9853 - val_loss: 0.4474 - val_acc: 0.8637
